A timeline about the different phases of the museums architecture and the public art around it, based on images pulled from Wikimedia Commons.

Created by: Ash	Dammeyer, 16 June 2025

In [1]:
COMMONS_CATEGORY = "Sprengel_Museum_Architecture_Timeline"

COMMONS_API = "https://commons.wikimedia.org/w/api.php"
WIKIDATA_API = "https://www.wikidata.org/w/api.php"

In [2]:
import requests, folium, time
from folium import plugins
from folium.plugins import BeautifyIcon, MarkerCluster



def get_files(category, limit=50):
    r = requests.get(COMMONS_API, params={
      "action":"query","format":"json",
      "list":"categorymembers","cmtitle":f"Category:{category}",
      "cmlimit":limit,"cmtype":"file","nocache":str(time.time())
    })
    return r.json()["query"]["categorymembers"]

def get_coords(title):
    r = requests.get(COMMONS_API, params={
      "action":"query","titles":title,"prop":"coordinates",
      "colimit":"1","format":"json","nocache":str(time.time())
    }).json()
    for p in r["query"]["pages"].values():
        if "coordinates" in p:
            return p["coordinates"][0]["lat"], p["coordinates"][0]["lon"]
    return None,None

def get_description(title):
    r = requests.get(COMMONS_API, params={
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "imageinfo",
        "iiprop": "extmetadata",
    }).json()
    for p in r["query"]["pages"].values():
        meta = p.get("imageinfo", [{}])[0].get("extmetadata", {})
        return meta.get("ImageDescription", {}).get("value", "No description available.")
    return "No description found."

def get_url(title):
    r = requests.get(COMMONS_API, params={
      "action":"query","titles":title,"prop":"imageinfo",
      "iiprop":"url","format":"json"
    }).json()
    for p in r["query"]["pages"].values():
        return p["imageinfo"][0]["url"]

def get_depicts_qid(title):
    """
    Return a list of Q-IDs from P180 (“depicts”) for the given Commons file title.
    If the file has no P180 statements, returns [].
    """
    # Step 1: Fetch page info to find pageid → MediaInfo ID
    r1 = requests.get(COMMONS_API, params={
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "info"
    })
    r1.raise_for_status()
    pages = r1.json().get("query", {}).get("pages", {})
    media_id = None
    for page_id, page in pages.items():
        if page.get("missing"):
            continue
        media_id = f"M{page_id}"
        break
    if not media_id:
        return []
    # Step 2: Use wbgetentities to get the structured data under "statements"
    r2 = requests.get(COMMONS_API, params={
        "action": "wbgetentities",
        "format": "json",
        "ids": media_id,
        "props": "claims"
    })
    r2.raise_for_status()
    entities = r2.json().get("entities", {})
    ent = entities.get(media_id, {})
    # Step 3: Extract P180 under ent["statements"]
    statements = ent.get("statements", {})
    qid = ''
    for claim in statements.get("P180", []):
        dv = claim.get("mainsnak", {}).get("datavalue", {})
        value = dv.get("value", {})
        qid = value.get("id")
    return qid

import requests

WIKIDATA_API = "https://www.wikidata.org/w/api.php"

def get_wikipedia_url_from_qid(qid, lang="en"):
    """
    Given a Wikidata Q-ID (e.g. "Q510144") and an ISO-lang code (default "en"),
    return the corresponding Wikipedia URL (e.g. "https://en.wikipedia.org/wiki/Brandenburg_Gate"),
    or None if no such sitelink exists.
    """
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "sitelinks",
        "sitefilter": f"{lang}wiki",  # only return the <lang>wiki sitelink
        "format": "json"
    }
    r = requests.get(WIKIDATA_API, params=params, timeout=10)
    r.raise_for_status()
    data = r.json().get("entities", {}).get(qid, {})
    sitelinks = data.get("sitelinks", {})
    entry = sitelinks.get(f"{lang}wiki")
    if not entry:
        return None
    # Some API versions include a 'url' field directly:
    url = entry.get("url")
    if url:
        return url
    # Otherwise, build it from the title
    title = entry.get("title", "").replace(" ", "_")
    return f"https://{lang}.wikipedia.org/wiki/{title}" if title else None



def get_geotagged_files(category, target=9, batch=50):
    files = get_files(category, limit=batch)
    locations = []
    for f in files:
        lat, lon = get_coords(f["title"])
        if lat is not None:
            locations.append((f["title"], lat, lon, get_url(f["title"]), get_description(f["title"]), get_depicts_qid(f["title"])))
            if len(locations) == target:
                break
        time.sleep(0.2)
    return locations

locations = get_geotagged_files(COMMONS_CATEGORY, target=9)


if not locations:
    print("No structured geocoordinates found.")
else:
    m = folium.Map(location=[locations[0][1], locations[0][2]], zoom_start=16, max_zoom=25)
    marker_cluster = MarkerCluster().add_to(m)

for idx, (name, lat, lon, img, desc, qid) in enumerate(locations, start=1):
    # clean up the label
    label = name.replace('File:', '').rsplit('.', 1)[0]
    
    # build the popup as before
    popup_html = f"""
    <h4 style="text-align: center; font-weight:bold">{label}</h4><br>
    <img src="{img}" style="
        width: 200px;      /* fixed square width */
        height: 200px;     /* fixed square height */
        object-fit: cover; /* crop/scale to fill */
        border-radius: 100px;
        display: block;
        margin: auto;
    ">
    """


    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_html, max_width=220),
        icon=BeautifyIcon(
            icon_shape='marker', number=idx, background_color='lightblue'
        ),
        tooltip=label
    ).add_to(marker_cluster)


folium.plugins.Fullscreen(
    position="topright",
    title="Vollbild",
    title_cancel="Schließen",
    force_separate_button=True,
).add_to(m)


m

In [3]:
##############
# INFO TABLE #
##############

# TABLE HEADER

print("""
```{=html}
      </br>
      <table class='table table-striped' id='mapTable'>
      <thead>
      <tr>
      <th style="width:5%">Nr.</th>
      <th style="width:35%"><b onclick="sortTable(0)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Name</th>
      <th><b onclick="sortTable(1)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Beschreibung</th>
      <th style="width:10%"><b onclick="sortTable(2)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
      <th style="width:5%"> </th> 
      </tr>
      </thead>
      <tbody>
```
      """)

# ITERATE THROUGH INFO LIST AGAIN
for idx, (name, lat, lon, img, desc, qid) in enumerate(locations, start=1):
  clean = name.replace('File:', '').rsplit('.', 1)[0]

  # GET WIKI LINK
  wiki = get_wikipedia_url_from_qid(qid, lang="en")

  # NUMBERING
  print(f"""
```{{=html}}
<tr>
<td><b>{idx}</b></td>
```
  """)

  # TITLE & DESCRIPTION 
  print(f"""
```{{=html}}
<td><span style='display:none'>{name}</span><a href='https://commons.wikimedia.org/wiki/{name}'>{clean}</a></td>
<td>{desc}</td> 
```
""")

  # COORDINATES 
  print(f"""
```{{=html}}
  <td>{lat},<br> {lon}</td>
```
""")
  
  # LINK TO RESONATOR IF Q-ID EXISTS
  if qid != '':
    print(f"""
```{{=html}}
<td><a href="https://reasonator.toolforge.org/?q={qid}&lang=mul"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Reasonator_logo_proposal.png/24px-Reasonator_logo_proposal.png"></a>
```
""")
  # LINK TO WIKIPEDIA PAGE IF ONE EXISTS
    if wiki:
      print(f"""
```{{=html}}
<br><a href="{wiki}"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/Wikipedia%27s_W.svg" height="24px"></a></td>
```    
""")
    else: 
      print("""
```{=html}
</td>
```
      """)
  else:
    print("""
```{=html}
<td></td>
```
    """)

  print("""
```{=html}
</tr>
```
""")
print("""
```{=html}
</tbody></table>
```
""")

# JAVASCRIPT FOR TABLE SORTING
print("""
<script>
function sortTable(n) {
  var table, rows, switching, i, x, y, shouldSwitch, dir, switchcount = 0;
  table = document.getElementById("mapTable");
  switching = true;
  // Set the sorting direction to ascending:
  dir = "asc";
  /* Make a loop that will continue until
  no switching has been done: */
  while (switching) {
    // Start by saying: no switching is done:
    switching = false;
    rows = table.rows;
    /* Loop through all table rows (except the
    first, which contains table headers): */
    for (i = 1; i < (rows.length - 1); i++) {
      // Start by saying there should be no switching:
      shouldSwitch = false;
      /* Get the two elements you want to compare,
      one from current row and one from the next: */
      x = rows[i].getElementsByTagName("TD")[n];
      y = rows[i + 1].getElementsByTagName("TD")[n];
      /* Check if the two rows should switch place,
      based on the direction, asc or desc: */
      if (dir == "asc") {
        if (x.innerHTML.toLowerCase() > y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      } else if (dir == "desc") {
        if (x.innerHTML.toLowerCase() < y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      }
    }
    if (shouldSwitch) {
      /* If a switch has been marked, make the switch
      and mark that a switch has been done: */
      rows[i].parentNode.insertBefore(rows[i + 1], rows[i]);
      switching = true;
      // Each time a switch is done, increase this count by 1:
      switchcount ++;
    } else {
      /* If no switching has been done AND the direction is "asc",
      set the direction to "desc" and run the while loop again. */
      if (switchcount == 0 && dir == "asc") {
        dir = "desc";
        switching = true;
      }
    }
  }
}
</script>
""") 


```{=html}
      </br>
      <table class='table table-striped' id='mapTable'>
      <thead>
      <tr>
      <th style="width:5%">Nr.</th>
      <th style="width:35%"><b onclick="sortTable(0)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Name</th>
      <th><b onclick="sortTable(1)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Beschreibung</th>
      <th style="width:10%"><b onclick="sortTable(2)" style="color: blue; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
      <th style="width:5%"> </th> 
      </tr>
      </thead>
      <tbody>
```
      

```{=html}
<tr>
<td><b>1</b></td>
```
  

```{=html}
<td><span style='display:none'>File:A corner view of the Sprengel Museum Hannover.jpg</span><a href='https://commons.wikimedia.org/wiki/File:A corner view of the Sprengel Museum Hannover.jpg'>A corner view of the Sprengel Museum Hannover</a></td>
<td>A corner view of the Sprengel Museum Hannover</td> 
```


```{=html}
  <td>52.362239,<br> 9.740075</td>
```


```{=html}
<td><a href="htt